# TP1 : Web scraping et APIs

## Imports

In [2]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des politiciens belges

In [3]:
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = Country of citizenship 
# Q31 = Belgium
# P39 = Position held 
# Q33126365 = Mayor of Brussels
# P569 = date of birth
# P570 = date of death

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P39 wd:Q33126365 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?dateBirth
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Mayor of Brussels\n")
print (results)


27 Mayor of Brussels

{'head': {'vars': ['person', 'personLabel', 'dateBirth', 'dateDeath']}, 'results': {'bindings': [{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q105605007'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Henri Joseph Van Langhenhoven'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1749-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16661300'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Louis Devos'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1757-07-24T00:00:00Z'}, 'dateDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1830-12-30T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2009106'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Nicolas-Jean Rouppe'}, 'da

## Obtenir la liste des politiciens de la ville de Bruxelles avec leurs dates de naissances


In [6]:
statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P39 wd:Q33126365 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?dateBirth
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

date_format = "%Y-%m-%dT%H:%M:%SZ"

for row in rows:
    try:
        dateBirth = dt.strptime(row['dateBirth']['value'], date_format)
    except ValueError:
        dateBirth = "????"
    try:
        personLabel = row['personLabel']['value']
    except ValueError: # unknown
        personLabel = "????"

    print(f"{row['personLabel']['value']} - {dateBirth.date()})")

Henri Joseph Van Langhenhoven - 1749-01-01)
Louis Devos - 1757-07-24)
Nicolas-Jean Rouppe - 1768-04-17)
Louis de Wellens van ten Meulenberg - 1772-08-06)
Charles-Joseph, 4th Duke d'Ursel - 1777-08-09)
François-Jean Wyns de Raucourt - 1779-11-06)
Joseph van der Linden d'Hooghvorst - 1782-06-24)
Guillaume Van Volxem - 1791-02-13)
Charles de Brouckère - 1796-01-18)
André-Napoléon Fontainas - 1807-12-23)
Felix Vanderstraeten - 1823-07-18)
Jules Anspach - 1829-07-20)
Émile de Mot - 1835-10-20)
Charles Buls - 1837-10-13)
Adolphe Max - 1869-12-30)
Jules Coelst - 1870-02-11)
Joseph Van De Meulebroeck - 1876-11-17)
Jan Grauls - 1887-04-19)
Lucien Cooremans - 1899-09-01)
Pierre Van Halteren - 1911-02-24)
Hervé Brouhon - 1924-06-18)
Michel Demaret - 1940-01-18)
François-Xavier de Donnea - 1941-04-29)
Freddy Thielemans - 1944-09-11)
Marion Lemesre - 1954-07-22)
Yvan Mayeur - 1960-01-24)
Philippe Close - 1971-03-18)


## Interroger une API pour enrichir les données des bulletins communaux de la ville de Bruxelles

# APIs: requêtes HTTP

## Imports

In [2]:
import json
import requests

## Utiliser GeoDB Cities pour obtenir des détails de Bruxelles, y compris ces coordonnées, sa population et son altitude au-dessus du niveau de la mer (si disponible).

http://geodb-cities-api.wirefreethought.com

In [6]:
# GeoDB Cities ne demande pas de paeiment et une tentative de cet API est offert

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q239"

#Code de Bruxelles se trouve sur https://www.wikidata.org/wiki/Q239

headers = {
	"X-RapidAPI-Key": "e91070c489msh147a45d61225d88p13437ejsn463e22266745",
	"X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"data":{"id":7918,"wikiDataId":"Q239","type":"CITY","city":"City of Brussels","name":"City of Brussels","country":"Belgium","countryCode":"BE","region":"Brussels-Capital Region","regionCode":"BRU","elevationMeters":null,"latitude":50.85045,"longitude":4.34878,"population":185103,"timezone":"Europe__Brussels","distance":null,"deleted":false,"placeType":"CITY"}}
